In [21]:
import os
import re
import numpy as np

# Define the base directory where the expansible log directories are located
for method in ["eac", "stkec", "trafficstream"]:
    print(" " * 50 + "*" * 10 + method +"*" * 10)
    # method = "stkec"  # expansible / retrained / fullretrained / static / trafficStream / stkec
    base_dir = f"./log/ENERGY-Wind/{method}"

    # Define the directories to parse
    directories = [f"{base_dir}-42", f"{base_dir}-43", f"{base_dir}-44", f"{base_dir}-45", f"{base_dir}-46"] # , f"{base_dir}-45", f"{base_dir}-46"

    # Define a regular expression to extract the metric values
    metric_pattern = re.compile(r"(\d+|Avg)\s+(MAE|RMSE|MAPE)\s+([\d\.\s]+)")
    total_time_pattern = re.compile(r"total time:\s+([\d\.]+)")

    # Initialize a dictionary to store metrics
    metrics = {
        "3": {"MAE": [], "RMSE": [], "MAPE": []},
        "6": {"MAE": [], "RMSE": [], "MAPE": []},
        "12": {"MAE": [], "RMSE": [], "MAPE": []},
        "Avg": {"MAE": [], "RMSE": [], "MAPE": []}
    }

    # List to store total times
    total_times = []

    # Function to parse the log file and extract the last column value of metric lines
    def parse_log_file(file_path):
        with open(file_path, "r") as file:
            lines = file.readlines()[-15:]
            for line in lines:
                match = metric_pattern.search(line)
                
                total_time_match = total_time_pattern.search(line)
                if match:
                    key, metric_type, values = match.groups()
                    values_list = list(map(float, values.split()))
                    last_value = values_list[-1]  # Extract the last value
                    if key in metrics:
                        metrics[key][metric_type].append(last_value)
                if total_time_match:
                    total_time = float(total_time_match.group(1))
                    total_times.append(total_time)


    # Process each directory and extract metrics
    for directory in directories:
        log_file_path = os.path.join(directory, f"{method}.log")
        if os.path.exists(log_file_path):
            parse_log_file(log_file_path)


    # Function to compute mean and variance for the metrics
    def compute_statistics(metric_data):
        metric_array = np.array(metric_data)
        mean = np.mean(metric_array)
        variance = np.std(metric_array)
        return mean, variance

    formatted_results = []

    for key in metrics:
        for metric_type in ["MAE", "RMSE", "MAPE"]:
            mean, variance = compute_statistics(metrics[key][metric_type])
            formatted_results.append(f"{mean:.2f}±{variance:.2f}\t")

    # Compute mean for total times
    time_mean, time_variance = compute_statistics(total_times)

    # Output the formatted results
    header = " "*22 + "3" + " " * 48 + "6" + " " * 46 + "12" + " " * 46 + "Avg" + " " * 27 + "Total Time"
    sub_header = "    {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}\t     {:<10}".format(
        "MAE", "RMSE", "MAPE", "MAE", "RMSE", "MAPE", "MAE", "RMSE", "MAPE", "MAE", "RMSE", "MAPE"
    )
    values_row = "{:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}  {:.2f}±{:.2f}".format(*formatted_results, time_mean,time_variance)

    print(header)
    print(sub_header)
    print(values_row)
    print('\n')

                                                  **********eac**********
                      3                                                6                                              12                                              Avg                           Total Time
    MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	     MAPE      
3.31±0.01	 3.65±0.01	 36.41±0.65	 3.35±0.01	 3.73±0.01	 37.16±0.52	 3.47±0.01	 3.94±0.00	 39.66±0.41	 3.37±0.01	 3.75±0.01	 37.55±0.55	  0.00±0.00


                                                  **********stkec**********
                      3                                                6                                              12                                              Avg                           Total Time
    MAE       	     RMSE      	     MAPE      	     MAE       	     RMSE      	     MAPE      	 